# Table `system-ranking-correlation-on-all-runs-with-unjudged`

In [1]:
import pandas as pd

df = pd.read_json('system-ranking-correlation-on-all-runs-with-unjudged.jsonl', lines=True, orient='records')
df

,corpus,method,kendall,pearson
0,Robust04,min-residual,0.936449,0.996886
1,Robust04,max-residual,0.188785,-0.268189
2,Robust04,condensed,0.924214,0.978275
3,Robust04,bs-run-and-pool-dependent,0.965989,0.995518
4,Robust04,bs-pool-dependent,0.910948,0.974793
5,Robust04,bs-run-dependent,0.943488,0.996562
6,CW09,min-residual,0.821017,0.959085
7,CW09,max-residual,-0.410917,-0.656288
8,CW09,condensed,0.609660,0.744426
9,CW09,bs-run-and-pool-dependent,0.716102,0.885316


In [2]:
df[['method', 'kendall', 'pearson']].groupby('method').mean().reset_index()

,method,kendall,pearson
0,bs-pool-dependent,0.778751,0.902698
1,bs-run-and-pool-dependent,0.831996,0.934996
2,bs-run-dependent,0.809627,0.927444
3,condensed,0.773125,0.870410
4,max-residual,-0.106483,-0.391417
5,min-residual,0.801314,0.931015


# Creation of Data for Table on `system-ranking-correlation-on-all-runs-with-unjudged`

In [1]:
%%time
import sys
sys.path.append('../python/')
import pandas as pd
from sklearn.metrics import mean_squared_error
import numpy as np
from reconstruction_evaluation import load_df_reconstruction, load_preprocessed_reconstruction_or_from_cache, load_df
import json
from statistics import mean
from scipy.stats import ttest_ind
from tqdm import tqdm


CPU times: user 707 ms, sys: 1.43 s, total: 2.14 s
Wall time: 20.1 s


In [2]:
def load_calculated_scores(cache_file='retrieval-effectiveness-scores-results.json'):
    dfs = {}
    corpora = {
        'Robust04': ['trec13'],
        'CW09': ['trec18', 'trec19', 'trec20', 'trec21'],
        'CW12': ['trec22', 'trec23']
    }

    # We follow the preprocessing steps of Zobel et al. and include only the top 75% of the runs to mitigate the effects of low-performing runs
    runs_to_include = {'Robust04': 82, 'CW09': 24, 'CW12': 22}

    for corpus, trecs in tqdm(corpora.items(), 'Corpora'):
        df_for_corpus = []
        for trec in trecs:
            df_for_corpus += [
                load_df_reconstruction(trec, runs_to_include[corpus], True, min_unjudged=0, max_unjudged=0.7)]
        dfs[corpus] = pd.concat(df_for_corpus)
    return dfs

def load_df_reconstruction(trec, num_runs_to_keep=100000, failsave=True, min_unjudged=0, max_unjudged=None):
    runs_to_keep = pd.read_json('../resources/processed/ndcg-at-10-effectiveness.jsonl', lines=True)
    runs_to_keep = runs_to_keep[runs_to_keep['position'] < num_runs_to_keep]
    runs_to_keep = set(runs_to_keep['run'].unique())
    return load_df(trec)

dfs = load_calculated_scores()

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4331/4331 [00:21<00:00, 202.28it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3416/3416 [00:16<00:00, 201.09it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2257/2257 [00:12<00:00, 187.70it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2074/2074 [00:10<00:00, 202.35it/s]

Corpora: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [09:18<00:00, 186.13s/it]


In [4]:
dfs['Robust04']


,,"(depth-10-incomplete, bs-run-dependent-1000-ndcg@10-ndcg@10)","(depth-10-complete, unjudged@10)","(depth-10-incomplete, unjudged@10)","(depth-20-complete, unjudged@10)","(depth-20-incomplete, unjudged@10)","(depth-10-complete, ndcg@10)","(depth-10-incomplete, ndcg@10)","(depth-20-complete, ndcg@10)","(depth-20-incomplete, ndcg@10)","(depth-10-complete, residual-ndcg@10-min)",...,"(depth-10-incomplete, gsd-upper-bound-0.05-condensed-ndcg@10-condensed-ndcg@10-gsd-upper-bound-0.05-condensed-ndcg@10-condensed-ndcg@10)","(depth-10-incomplete, always-condensed-ndcg@10-condensed-ndcg@10-always-condensed-ndcg@10-condensed-ndcg@10)","(depth-10-incomplete, always-1)","(depth-10-incomplete, always-0)","(depth-10-incomplete, gsd-lower-bound-0.01-ndcg@10-ndcg@10-gsd-lower-bound-0.01-ndcg@10-ndcg@10)","(depth-10-incomplete, gsd-lower-bound-0.05-ndcg@10-ndcg@10-gsd-lower-bound-0.05-ndcg@10-ndcg@10)","(depth-10-incomplete, gsd-upper-bound-0.01-ndcg@10-ndcg@10-gsd-upper-bound-0.01-ndcg@10-ndcg@10)","(depth-10-incomplete, gsd-upper-bound-0.05-ndcg@10-ndcg@10-gsd-upper-bound-0.05-ndcg@10-ndcg@10)","(depth-10-incomplete, always-ndcg@10-ndcg@10-always-ndcg@10-ndcg@10)",run
run,,,,,,,,,,,,,,,,,,,,,,
JuruDes,0,"{""301"": [0.07336392209936005, 0.22009176629808...","{""301"": 0.0, ""302"": 0.0, ""303"": 0.0, ""304"": 0....","{""301"": 0.2, ""302"": 0.0, ""303"": 0.0, ""304"": 0....","{""301"": 0.0, ""302"": 0.0, ""303"": 0.0, ""304"": 0....","{""301"": 0.1, ""302"": 0.0, ""303"": 0.0, ""304"": 0....","{""301"": 0.22009176629808017, ""302"": 0.77215962...","{""301"": 0, ""302"": 0.7721596254947202, ""303"": 0...","{""301"": 0.22009176629808017, ""302"": 0.77215962...","{""301"": 0.22009176629808017, ""302"": 0.77215962...","{""301"": 0.22009176629808017, ""302"": 0.77215962...",...,"{""301"": 0.12987501170000001, ""302"": 0.77215962...","{""301"": 0.12987501170000001, ""302"": 0.77215962...","{""301"": 1.0, ""302"": 1.0, ""303"": 1.0, ""308"": 1....","{""301"": 0.0, ""302"": 0.0, ""303"": 0.0, ""308"": 0....","{""301"": 0.0, ""302"": 0.7721596255000001, ""303"":...","{""301"": 0.0, ""302"": 0.7721596255000001, ""303"":...","{""301"": 0.0, ""302"": 1.0, ""303"": 0.1749434282, ...","{""301"": 0.0, ""302"": 0.8416539918, ""303"": 0.095...","{""301"": 0.0, ""302"": 0.7721596255000001, ""303"":...",JuruDes
JuruDesAggr,0,"{""301"": [0.15176219107803537, 0.15176219107803...","{""301"": 0.0, ""302"": 0.0, ""303"": 0.0, ""304"": 0....","{""301"": 0.2, ""302"": 0.0, ""303"": 0.0, ""304"": 0....","{""301"": 0.0, ""302"": 0.0, ""303"": 0.0, ""304"": 0....","{""301"": 0.1, ""302"": 0.0, ""303"": 0.0, ""304"": 0....","{""301"": 0.3557772116892465, ""302"": 0.772159625...","{""301"": 0.13568544539116628, ""302"": 0.77215962...","{""301"": 0.3557772116892465, ""302"": 0.772159625...","{""301"": 0.3557772116892465, ""302"": 0.772159625...","{""301"": 0.23690530871965634, ""302"": 0.77215962...",...,"{""301"": 0.2180164145, ""302"": 0.772159625500000...","{""301"": 0.2180164145, ""302"": 0.772159625500000...","{""301"": 1.0, ""302"": 1.0, ""303"": 1.0, ""308"": 1....","{""301"": 0.0, ""302"": 0.0, ""303"": 0.0, ""308"": 0....","{""301"": 0.1356854454, ""302"": 0.772159625500000...","{""301"": 0.1356854454, ""302"": 0.772159625500000...","{""301"": 0.2713708908, ""302"": 1.0, ""303"": 0.174...","{""301"": 0.1492539899, ""302"": 0.849375588, ""303...","{""301"": 0.1356854454, ""302"": 0.772159625500000...",JuruDesAggr
JuruDesLaMd,0,"{""301"": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.22009...","{""301"": 0.0, ""302"": 0.0, ""303"": 0.0, ""304"": 0....","{""301"": 0.2, ""302"": 0.0, ""303"": 0.0, ""304"": 0....","{""301"": 0.0, ""302"": 0.0, ""303"": 0.0, ""304"": 0....","{""301"": 0.1, ""302"": 0.0, ""303"": 0.0, ""304"": 0....","{""301"": 0.22009176629808017, ""302"": 0.77215962...","{""301"": 0, ""302"": 0.7721596254947202, ""303"": 0...","{""301"": 0.22009176629808017, ""302"": 0.77215962...","{""301"": 0.22009176629808017, ""302"": 0.77215962...","{""301"": 0.22009176629808

In [46]:
def to_mean(i):
    if str(i) == 'nan':
        return float('nan')
    
    i = list(json.loads(i).values())
    
    if type(i[0]) == list:
        i = BootstrappingBySelectingMostLikelyDataPoint('name-is-ignored-for-ad-hoc-prediction').predict(i)
    
    return mean(i)

dfs['Robust04']['ndcg@10(ground_truth)'] = dfs['Robust04'][('depth-10-complete', 'ndcg@10')].apply(to_mean)
dfs['Robust04']['ndcg@10(min-residual)'] = dfs['Robust04'][('depth-10-incomplete', 'residual-ndcg@10-min')].apply(to_mean)
dfs['Robust04']['ndcg@10(max-residual)'] = dfs['Robust04'][('depth-10-incomplete', 'residual-ndcg@10-max')].apply(to_mean)
dfs['Robust04']['ndcg@10(condensed)'] = dfs['Robust04'][('depth-10-incomplete', 'condensed-ndcg@10')].apply(to_mean)

dfs['Robust04']['ndcg@10(bs-run-and-pool-dependent)'] = dfs['Robust04'][('depth-10-incomplete', 'bs-run-and-pool-dependent-1000-ndcg@10-ndcg@10')].apply(to_mean)
dfs['Robust04']['ndcg@10(bs-pool-dependent)'] = dfs['Robust04'][('depth-10-incomplete', 'bs-pool-dependent-1000-ndcg@10-ndcg@10')].apply(to_mean)
dfs['Robust04']['ndcg@10(bs-run-dependent)'] = dfs['Robust04'][('depth-10-incomplete', 'bs-run-dependent-1000-ndcg@10-ndcg@10')].apply(to_mean)



#
#residual-ndcg@10-max
#condensed-ndcg@10
#''
#'bs-pool-dependent-1000-ndcg@10-ndcg@10'
#'bs-run-dependent-1000-ndcg@10-ndcg@10'

In [48]:
dfs['Robust04'][['ndcg@10(ground_truth)', 'ndcg@10(ground_truth)']].corr('kendall')

,ndcg@10(ground_truth),ndcg@10(ground_truth)
ndcg@10(ground_truth),1.0,1.0
ndcg@10(ground_truth),1.0,1.0


In [62]:
def correlation(df, to, corr):
    return df[['ndcg@10(ground_truth)', f'ndcg@10({to})']].corr(corr).iloc[0][f'ndcg@10({to})']

df_corr = []
for corpus in dfs.keys():
    df = dfs[corpus].copy()
    df['ndcg@10(ground_truth)'] = df[('depth-10-complete', 'ndcg@10')].apply(to_mean)
    df['ndcg@10(min-residual)'] = df[('depth-10-incomplete', 'residual-ndcg@10-min')].apply(to_mean)
    df['ndcg@10(max-residual)'] = df[('depth-10-incomplete', 'residual-ndcg@10-max')].apply(to_mean)
    df['ndcg@10(condensed)'] = df[('depth-10-incomplete', 'condensed-ndcg@10')].apply(to_mean)

    df['ndcg@10(bs-run-and-pool-dependent)'] = df[('depth-10-incomplete', 'bs-run-and-pool-dependent-1000-ndcg@10-ndcg@10')].apply(to_mean)
    df['ndcg@10(bs-pool-dependent)'] = df[('depth-10-incomplete', 'bs-pool-dependent-1000-ndcg@10-ndcg@10')].apply(to_mean)
    df['ndcg@10(bs-run-dependent)'] = df[('depth-10-incomplete', 'bs-run-dependent-1000-ndcg@10-ndcg@10')].apply(to_mean)
    
    for method in ['min-residual', 'max-residual', 'condensed', 'bs-run-and-pool-dependent', 'bs-pool-dependent', 'bs-run-dependent']:
        df_corr += [{
            'corpus': corpus,
            'method': method,
            'kendall': correlation(df, method, 'kendall'),
            'pearson': correlation(df, method, 'pearson'),
        }]

df_corr = pd.DataFrame(df_corr)
df_corr.to_json('system-ranking-correlation-on-all-runs-with-unjudged.jsonl', lines=True, orient='records')

df_corr

,corpus,method,kendall,pearson
0,Robust04,min-residual,0.936449,0.996886
1,Robust04,max-residual,0.188785,-0.268189
2,Robust04,condensed,0.924214,0.978275
3,Robust04,bs-run-and-pool-dependent,0.965989,0.995518
4,Robust04,bs-pool-dependent,0.910948,0.974793
5,Robust04,bs-run-dependent,0.943488,0.996562
6,CW09,min-residual,0.821017,0.959085
7,CW09,max-residual,-0.410917,-0.656288
8,CW09,condensed,0.609660,0.744426
9,CW09,bs-run-and-pool-dependent,0.716102,0.885316
